In [2]:
import pandas as pd
import utils
import plotly.graph_objects as go
import instrument

In [3]:
pair = "CAD_CHF"
granularity = "H1"
ma_list = [16,64]
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [4]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [5]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [6]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2023-04-24T18:00:00.000000000Z,0.65588,0.65590,0.65532,0.65568,0.657024,0.660572
1,2023-04-24T19:00:00.000000000Z,0.65566,0.65612,0.65566,0.65604,0.656872,0.660419
2,2023-04-24T20:00:00.000000000Z,0.65603,0.65607,0.65546,0.65554,0.656708,0.660261
3,2023-04-24T21:00:00.000000000Z,0.65574,0.65602,0.65546,0.65560,0.656539,0.660106
4,2023-04-24T22:00:00.000000000Z,0.65558,0.65564,0.65540,0.65560,0.656398,0.659950


In [7]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return 0

In [8]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()

In [9]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
66,2023-04-27T12:00:00.000000000Z,0.65634,0.65850,0.65519,0.65787,0.654308,0.654113,0.000195,-0.000039,1
146,2023-05-02T20:00:00.000000000Z,0.65532,0.65563,0.65520,0.65558,0.658992,0.659198,-0.000206,0.000152,-1
199,2023-05-05T01:00:00.000000000Z,0.65472,0.65473,0.65420,0.65439,0.653466,0.653211,0.000255,-0.000006,1
273,2023-05-10T03:00:00.000000000Z,0.66506,0.66512,0.66478,0.66486,0.665484,0.665545,-0.000061,0.000119,-1
303,2023-05-11T09:00:00.000000000Z,0.66723,0.66738,0.66648,0.66692,0.665543,0.665529,0.000015,-0.000114,1


In [10]:
df_trades["DELTA"] = ( df_trades.mid_c.diff() / i_pair.pipLocation).shift(-1)
df_trades["GAIN"] =df_trades["DELTA"] * df_trades["IS_TRADE"]

In [17]:
from dateutil.parser import *
df_trades["time"] = [parse(x) for x in df_trades.time]

In [19]:
df_trades["DURATION"] = df_trades.time.diff().shift(-1)

In [25]:
df_trades["DURATION"] = [x.total_seconds() / 3600 for x in df_trades.DURATION]

In [27]:
df_trades[['time', 'DURATION']].tail()

,time,DURATION
3872,2023-12-06 05:00:00+00:00,5.0
3877,2023-12-06 10:00:00+00:00,2.0
3879,2023-12-06 12:00:00+00:00,5.0
3884,2023-12-06 17:00:00+00:00,34.0
3918,2023-12-08 03:00:00+00:00,NaN


In [20]:
df_trades.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73 entries, 66 to 3918
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype                  
---  ------     --------------  -----                  
 0   time       73 non-null     datetime64[ns, tzutc()]
 1   mid_o      73 non-null     float64                
 2   mid_h      73 non-null     float64                
 3   mid_l      73 non-null     float64                
 4   mid_c      73 non-null     float64                
 5   MA_16      73 non-null     float64                
 6   MA_64      73 non-null     float64                
 7   DIFF       73 non-null     float64                
 8   DIFF_PREV  73 non-null     float64                
 9   IS_TRADE   73 non-null     int64                  
 10  DELTA      72 non-null     float64                
 11  GAIN       72 non-null     float64                
 12  DURATION   72 non-null     timedelta64[ns]        
dtypes: datetime64[ns, tzutc()](1), float64(10), int64(1), 

In [28]:
df_all = pd.read_pickle("all_trades.pkl")

In [30]:
df_all.describe()

,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,MASHORT,MALONG,DURATION
count,64681.000000,64681.000000,6.468100e+04,64681.000000,64681.000000,64681.000000,64681.000000,64681.000000,64681.000000
mean,39.435939,0.000066,-1.310270e-04,0.001005,3.198375,-0.330844,11.776132,60.437161,51.248342
std,64.091972,0.021614,2.160798e-02,1.000007,72.715599,72.785154,12.508732,64.011155,85.720500
min,0.518710,-0.427004,-4.495156e-01,-1.000000,-800.900000,-524.300000,4.000000,8.000000,1.000000
25%,0.878580,-0.000192,-1.925000e-04,-1.000000,-20.500000,-28.200000,4.000000,16.000000,6.000000
50%,1.329440,0.000000,-3.125000e-07,1.000000,0.900000,-9.600000,8.000000,32.000000,18.000000
75%,87.516000,0.000190,1.900000e-04,1.000000,23.300000,8.000000,16.000000,96.000000,63.000000
max,188.383000,0.465750,4.523203e-01,1.000000,1251.800000,1251.800000,64.000000,256.000000,1308.000000


In [12]:
df_trades["GAIN"].sum()

395.70000000000533

In [13]:
df_plot = df_ma.iloc[0:24].copy()

In [14]:
df_plot.shape

(24, 10)

In [15]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()